In [2]:
import pandas as pd
from pulp import *
solver = PULP_CBC_CMD(msg=False)

## 1. Análise e tratamento da planilha
A planilha utilizada foi a [Tabela Brasileira de Composição de Alimentos - 4a edição](https://www.nepa.unicamp.br/taco/tabela.php?ativo=tabela)

Para o escopo do projeto, serão analisados somente os macronutrientes proteína, carboidratos e gorduras (lipídeos); e os micronutrientes vitaminas A e C, minerais cálcio e ferro.

As demais colunas serão removidas.

In [3]:
df = pd.read_excel('taco3.xls')

def	tratar_tabela(df):
	# Elimina colunas não usadas
	df = df.drop(['Unnamed: 2', 'Unnamed: 7'] 
				 	+ df.columns[9:11].tolist() 
					+ df.columns[12:16].tolist() 
					+ df.columns[17:21].tolist() 
					+ df.columns[22:28].tolist(), axis=1)
					
	# Renomeia colunas
	df.columns = ['Número do alimento', 'Descrição', 'Energia (kcal)', 
					'Energia (kJ)', 'Proteína (g)', 'Gorduras (g)', 
					'Carboidrato (g)', 'Cálcio (mg)', 'Ferro (mg)', 
					'Vitamina A (mcg)', 'Vitamica C (mg)']

	# Elimina linhas redundantes e reseta index
	df = df.drop(index=[0, 1], axis=0)
	df = df.reset_index(drop=True)
	df = df.fillna(0)
	return df

df = tratar_tabela(df)

df

,Número do alimento,Descrição,Energia (kcal),Energia (kJ),Proteína (g),Gorduras (g),Carboidrato (g),Cálcio (mg),Ferro (mg),Vitamina A (mcg),Vitamica C (mg)
0,Cereais e derivados,0,0,0,0,0,0,0,0,0,0
1,1,"Arroz, integral, cozido",123.534893,516.86999,2.58825,1.000333,25.80975,5.204,0.262,0,0
2,2,"Arroz, integral, cru",359.678002,1504.892761,7.323286,1.864833,77.450714,7.818,0.948333,0,0
3,3,"Arroz, tipo 1, cozido",128.258486,536.633504,2.520817,0.227,28.05985,3.544333,0.076667,0,0
4,4,"Arroz, tipo 1, cru",357.789273,1496.990319,7.15854,0.335,78.759543,4.414333,0.677747,0,0
...,...,...,...,...,...,...,...,...,...,...,...
688,††,"Teores alcoólicos (g/100g): ¹ Cana, aguardente...",0,0,0,0,0,0,0,0,0
689,†††,Abreviações: g: grama; mg: micrograma; kcal: k...,0,0,0,0,0,0,0,0,0
690,††††,Limites de Quantificação: a) composição centes...,0,0,0,0,0,0,0,0,0
691,0,Valores correspondentes à somatória do resulta...,0,0,0,0,0,0,0,0,0


In [35]:
df1 = df.loc[df['Número do alimento'] != 0]

#Cereais e derivados
linhas_selecionadas = df1.loc[1:63]
df_cereais = pd.DataFrame(linhas_selecionadas)

#Verduras, hortaliças e derivados
linhas_selecionadas = df1.loc[72:179]
df_verduras = pd.DataFrame(linhas_selecionadas)

#Frutas e derivados
linhas_selecionadas = df1.loc[182:286]
df_frutas = pd.DataFrame(linhas_selecionadas)

#Gorduras e oleos
linhas_selecionadas = df1.loc[289:302]
df_gorduras = pd.DataFrame(linhas_selecionadas)

#Pescados e frutos do mar
linhas_selecionadas = df1.loc[306:361]
df_pescados = pd.DataFrame(linhas_selecionadas)

#Carnes e derivados
linhas_selecionadas = df1.loc[364:498]
df_carnes = pd.DataFrame(linhas_selecionadas)

#Leites e derivados
linhas_selecionadas = df1.loc[501:527]
df_leites = pd.DataFrame(linhas_selecionadas)

#Bebidas e derivados
linhas_selecionadas = df1.loc[530:543]
df_bebidas = pd.DataFrame(linhas_selecionadas)

#Ovos e derivados
linhas_selecionadas = df1.loc[546:553]
df_ovos = pd.DataFrame(linhas_selecionadas)

#Produtos acucarados
linhas_selecionadas = df1.loc[554:576]
df_acucarados = pd.DataFrame(linhas_selecionadas)

#Miscelaneas
linhas_selecionadas = df1.loc[579:588]
df_miscelaneas = pd.DataFrame(linhas_selecionadas)

#Alimentos industrializados
linhas_selecionadas = df1.loc[590:597]
df_industrializados = pd.DataFrame(linhas_selecionadas)

#Alimentos preparados
linhas_selecionadas = df1.loc[601:635]
df_preparados = pd.DataFrame(linhas_selecionadas)

#Leguminosas e derivados
linhas_selecionadas = df1.loc[638:670]
df_leguminosas = pd.DataFrame(linhas_selecionadas)

#Nozes e sementes
linhas_selecionadas = df1.loc[673:684]
df_nozes = pd.DataFrame(linhas_selecionadas)


In [ ]:
#depois quero fazer de todas bonitinho (ignorar)
import matplotlib.pyplot as plt

quantidade_alimentos = df_cereais.shape[0]

df_carnes.plot.scatter(x='Proteína (g)', y='Número do alimento')
plt.xlabel ('Proteína (g)')
plt.ylabel ('Número do alimento')
plt.show ()

## 2. Estudo das necessidades nutricionais dos indivíduos

Criando um dataframe com as necessidades nutricionais diárias dos indivíduos a partir da tabela passada na descrição do projeto.



In [5]:
lista_pessoas = [
	{'Sexo': 'Homem', 'Peso': 50.0, 'Proteínas': 40.0, 'Carboidratos': 200.0, 'Gorduras': 50.0, 'Vitamina A': 900.0, 'Vitamina C': 90.0, 'Cálcio': 1000.0, 'Ferro': 8.0},
	{'Sexo': 'Homem', 'Peso': 60.0, 'Proteínas': 48.0, 'Carboidratos': 240.0, 'Gorduras': 60.0, 'Vitamina A': 900.0, 'Vitamina C': 90.0, 'Cálcio': 1000.0, 'Ferro': 8.0},
	{'Sexo': 'Homem', 'Peso': 70.0, 'Proteínas': 56.0, 'Carboidratos': 280.0, 'Gorduras': 70.0, 'Vitamina A': 900.0, 'Vitamina C': 90.0, 'Cálcio': 1000.0, 'Ferro': 8.0},
	{'Sexo': 'Homem', 'Peso': 80.0, 'Proteínas': 64.0, 'Carboidratos': 320.0, 'Gorduras': 80.0, 'Vitamina A': 900.0, 'Vitamina C': 90.0, 'Cálcio': 1000.0, 'Ferro': 8.0},
	{'Sexo': 'Homem', 'Peso': 90.0, 'Proteínas': 72.0, 'Carboidratos': 360.0, 'Gorduras': 90.0, 'Vitamina A': 900.0, 'Vitamina C': 90.0, 'Cálcio': 1000.0, 'Ferro': 8.0},
	{'Sexo': 'Homem', 'Peso': 100.0, 'Proteínas': 80.0, 'Carboidratos': 400.0, 'Gorduras': 100.0, 'Vitamina A': 900.0, 'Vitamina C': 90.0, 'Cálcio': 1000.0, 'Ferro': 8.0},
	{'Sexo': 'Mulher', 'Peso': 50.0, 'Proteínas': 40.0, 'Carboidratos': 200.0, 'Gorduras': 50.0, 'Vitamina A': 700.0, 'Vitamina C': 75.0, 'Cálcio': 1000.0, 'Ferro': 18.0},
	{'Sexo': 'Mulher', 'Peso': 55.0, 'Proteínas': 43.0, 'Carboidratos': 215.0, 'Gorduras': 55.0, 'Vitamina A': 700.0, 'Vitamina C': 75.0, 'Cálcio': 1000.0, 'Ferro': 18.0},
	{'Sexo': 'Mulher', 'Peso': 60.0, 'Proteínas': 46.0, 'Carboidratos': 230.0, 'Gorduras': 60.0, 'Vitamina A': 700.0, 'Vitamina C': 75.0, 'Cálcio': 1000.0, 'Ferro': 18.0},
	{'Sexo': 'Mulher', 'Peso': 65.0, 'Proteínas': 49.0, 'Carboidratos': 245.0, 'Gorduras': 65.0, 'Vitamina A': 700.0, 'Vitamina C': 75.0, 'Cálcio': 1000.0, 'Ferro': 18.0},
	{'Sexo': 'Mulher', 'Peso': 70.0, 'Proteínas': 52.0, 'Carboidratos': 260.0, 'Gorduras': 70.0, 'Vitamina A': 700.0, 'Vitamina C': 75.0, 'Cálcio': 1000.0, 'Ferro': 18.0},
]

df_necessidades = pd.DataFrame(lista_pessoas)
df_necessidades

,Sexo,Peso,Proteínas,Carboidratos,Gorduras,Vitamina A,Vitamina C,Cálcio,Ferro
0,Homem,50.0,40.0,200.0,50.0,900.0,90.0,1000.0,8.0
1,Homem,60.0,48.0,240.0,60.0,900.0,90.0,1000.0,8.0
2,Homem,70.0,56.0,280.0,70.0,900.0,90.0,1000.0,8.0
3,Homem,80.0,64.0,320.0,80.0,900.0,90.0,1000.0,8.0
4,Homem,90.0,72.0,360.0,90.0,900.0,90.0,1000.0,8.0
5,Homem,100.0,80.0,400.0,100.0,900.0,90.0,1000.0,8.0
6,Mulher,50.0,40.0,200.0,50.0,700.0,75.0,1000.0,18.0
7,Mulher,55.0,43.0,215.0,55.0,700.0,75.0,1000.0,18.0
8,Mulher,60.0,46.0,230.0,60.0,700.0,75.0,1000.0,18.0
9,Mulher,65.0,49.0,245.0,65.0,700.0,75.0,1000.0,18.0


## 3. Desenvolvimento do software
Como a abordagem de otimização é livre. Foram escolhidas as seguintes formas de dietas que atinjam as necessidades nutricionais de forma otimizada:
1. Menor quantidade de kcal (perda de peso)

#### Explicando o código da função perda_peso
1. Remover strings em células das colunas dos nutrientes para evitar erros no PuLP
2. Método zip: O método zip em Python é uma função embutida que combina elementos de duas ou mais sequências (listas, tuplas, etc.) em uma sequência de tuplas em pares correspondentes. Combina-se, então, o nome do alimento com seu respectivo nutriente.
3. O método dict transforma todas essas tuplas em um dicionário em que o nome do alimento é a chave e o valor do nutriente é o valor.
4. LpVariable.dicts cria uma lista de variáveis de decisão para cada alimento. Por exemplo, se passássemos a lista `['item1', 'item2', 'item3']`, criaríamos as variáveis `x_item1, x_item2 e x_item3`.
5. A categoria `binary` assume apenas dois valores possíveis: 0 e 1. Se uma variável for 0, ela não será escolhida para a dieta. 

In [54]:
def perda_peso(df, df_necessidades, sexo, peso):
	# Transforma as linhas dos nutrientes que são strings em 0 para evitar erros no pulp
	colunas_remover_str = df.drop('Descrição', axis=1).columns
	df[colunas_remover_str] = df[colunas_remover_str].applymap(lambda x: 0 if isinstance(x, str) else x)

	# Cria dicionários dos alimentos e seus respectivos nutrientes
	alimentos = df['Descrição'].tolist()
	kcal = dict(zip(alimentos, df['Energia (kcal)']))
	prt = dict(zip(alimentos, df['Proteína (g)']))
	gord = dict(zip(alimentos, df['Gorduras (g)']))
	carb = dict(zip(alimentos, df['Carboidrato (g)']))
	calc = dict(zip(alimentos, df['Cálcio (mg)']))
	ferro = dict(zip(alimentos, df['Ferro (mg)']))
	vita = dict(zip(alimentos, df['Ferro (mg)']))
	vitc = dict(zip(alimentos, df['Vitamica C (mg)']))

	# Encontra a linha de interesse no df e salva-a no df "cliente"
	sexo_col = df_necessidades['Sexo'] == sexo
	peso_col = df_necessidades['Peso'] == peso
	cliente = df_necessidades[sexo_col & peso_col].reset_index(drop=True)

	# Instância do problema
	prob = LpProblem("perda_de_peso", LpMinimize)

	# Cria um dicionário com as variáveis de decisão
	x = LpVariable.dicts('x', alimentos, cat=LpBinary)

	# Função objetivo - a soma das calorias de cada variável (alimento) escolhida
	prob += lpSum([kcal[i] * x[i] for i in alimentos])

	# Restrições
	prob += lpSum([prt[i] * x[i] for i in alimentos]) >= cliente.loc[0, 'Proteínas']
	prob += lpSum([gord[i] * x[i] for i in alimentos]) >= cliente.loc[0, 'Gorduras']
	prob += lpSum([carb[i] * x[i] for i in alimentos]) >= cliente.loc[0, 'Carboidratos']
	prob += lpSum([calc[i] * x[i] for i in alimentos]) >= cliente.loc[0, 'Cálcio']
	prob += lpSum([ferro[i] * x[i] for i in alimentos]) >= cliente.loc[0, 'Ferro']
	prob += lpSum([vita[i] * x[i] for i in alimentos]) >= cliente.loc[0, 'Vitamina A']
	prob += lpSum([vitc[i] * x[i] for i in alimentos]) >= cliente.loc[0, 'Vitamina C']

	prob.solve(solver)
	print("Status:", LpStatus[prob.status])
	for alimento in alimentos:
		if x[alimento].value() == 1:
			print(alimento)

	print("Total de calorias:", value(prob.objective))

perda_peso(df, df_necessidades, 'Homem', 60)

Status: Optimal
Arroz, integral, cozido
Arroz, integral, cru
Arroz, tipo 1, cru
Arroz, tipo 2, cru
Aveia, flocos, crua
Biscoito, doce, maisena
Biscoito, doce, recheado com chocolate
Biscoito, doce, recheado com morango
Biscoito, doce, wafer, recheado de chocolate
Biscoito, doce, wafer, recheado de morango
Biscoito, salgado, cream cracker
Bolo, mistura para
Bolo, pronto, aipim
Bolo, pronto, chocolate
Bolo, pronto, coco
Bolo, pronto, milho
Canjica, branca, crua
Cereais, milho, flocos, com sal
Cereais, milho, flocos, sem sal
Cereais, mingau, milho, infantil
Cereais, mistura para vitamina, trigo, cevada e aveia
Cereal matinal, milho
Cereal matinal, milho, açúcar
Creme de arroz, pó
Creme de milho, pó
Curau, milho verde
Curau, milho verde, mistura para
Farinha, de arroz, enriquecida
Farinha, de centeio, integral
Farinha, de milho, amarela
Farinha, de rosca
Farinha, de trigo
Farinha, láctea, de cereais
Lasanha, massa fresca, cozida
Lasanha, massa fresca, crua
Macarrão, instantâneo
Macarrão, t

In [ ]:
def ganha_massa (df, df_necessidades, sexo, peso):
    # Transforma as linhas dos nutrientes que são strings em 0 para evitar erros no pulp
	colunas_remover_str = df.drop('Descrição', axis=1).columns
	df[colunas_remover_str] = df[colunas_remover_str].applymap(lambda x: 0 if isinstance(x, str) else x)
	
    
	# Cria dicionários dos alimentos e seus respectivos nutrientes
	cereais = df_cereais['Descrição'].tolist()
	cprt = dict(zip(cereais, df_cereais['Proteína (g)']))

	verduras = df_verduras['Descrição'].tolist()
	vprt = dict(zip(verduras, df_verduras['Proteína (g)']))

	frutas = df_frutas['Descrição'].tolist()
	fprt = dict(zip(frutas, df_frutas['Proteína (g)']))

	gorduras = df_gorduras['Descrição'].tolist()
	gprt = dict(zip(gorduras, df_gorduras['Proteína (g)']))

	carnes = df_carnes['Descrição'].tolist()
	caprt = dict(zip(carnes, df_carnes['Proteína (g)']))

	# Definiçã do problema
	prob = pulp.LpProblem('Ganho de massa', pulp.LpMaximiize)

	# Variáveis de decisão